In [3]:
import os
import pandas as pd
import parse
from transliterate import translit

labels = "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets.txt"
additional_labels =  "/home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets_additional.txt"


In [19]:
p = parse.compile("users/{username}{whitespace:s}{language}\n")
userdict = dict()
with open(labels, "r") as labelfile:
    for line in labelfile.readlines():
        line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]
with open(additional_labels, "r") as labelfile:
    for line in labelfile.readlines():
        #line = translit(line, "sr", reversed=True)
        found = p.parse(line)
        userdict[found["username"]] = found["language"]



In [20]:
set([value for key, value in userdict.items()])

{'bs', 'hr', 'me', 'sr'}

In [21]:
twitter_data_dir = "/home/peterr/macocu/Varieties/BCMS/users"
datafiles = os.listdir(twitter_data_dir)

missing = [filename for filename in datafiles if filename not in userdict.keys()]
len(missing)

386

In [22]:
len(datafiles)

1000

In [23]:
len(userdict.keys())

614

In [24]:
!wc -l /home/peterr/macocu/Varieties/BCMS/Denis*

  490 /home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets.txt
  124 /home/peterr/macocu/Varieties/BCMS/Denis_hrsrTweets_additional.txt
  614 total


In [25]:
present_languages = set([userdict.get(user, None) for user in datafiles])
present_languages

{None, 'bs', 'hr', 'me', 'sr'}

# Constructing a dataset

In [117]:
from bs4 import BeautifulSoup
from typing import List
def read_user_tweets(filename: str) -> List[str]:
    with open(filename, "r") as f:
        content = "<data>"+f.read()+"<\data>"
    data = BeautifulSoup(content, "xml")
    tweets = data.findAll("tweet")
    return [tweet.find("text").text for tweet in tweets]
import re
def remove_mentions(tweet: str) -> str:
    re_pattern = r"(^|[^@\w])@(\w{1,15})\b"
    return re.sub(re_pattern, " ", tweet)
def remove_URL(tweet: str) -> str:
    re_pattern = r"(https?:\/\/|\b)(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)"
    return re.sub(re_pattern, " ", tweet)
def remove_hashtags(tweet: str) -> str:
    re_pattern = r"#\w*[a-zA-Z]\w*"
    return re.sub(re_pattern, " ", tweet)
def remove_retweet_tag(tweet: str) -> str:
    re_pattern = r"^RT @(\w{1,15}):"
    return re.sub(re_pattern, "", tweet)

In [112]:
dataset = dict()
for user, language in userdict.items():
    filename = os.path.join(twitter_data_dir, user)
    tweets = read_user_tweets(filename)
    preprocess = lambda s: remove_hashtags(remove_URL(remove_mentions(remove_retweet_tag(s))))
    corrected_tweets = list(map(preprocess, tweets))
    dataset[user] = {"language": language, "tweets": corrected_tweets }

In [114]:
tweets_dataset_filename = "/home/peterr/macocu/taskB/data/interim/twitter.json"
import json
with open(tweets_dataset_filename, "w") as f:
    json.dump(dataset, f)

In [119]:
s = "RT @BegsDemijan: i nije bitno gde ja idem, nego da li želiš da pođeš sa mnom"
remove_retweet_tag(s)

' i nije bitno gde ja idem, nego da li želiš da pođeš sa mnom'